In [38]:
import tensorflow as tf
import cv2
import json
import numpy as np
import math
import os
from pandas import DataFrame
import pandas as pd
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from glob import glob
from tqdm import tqdm

In [39]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
SOURCE = '/content/gdrive/MyDrive/PART4.2/videos'
with open('/content/gdrive/MyDrive/PART4.2/KBS/ASS2/imagenet_class_index.json', 'r') as f:
    imagenet_classes = json.load(f)
imagenet_classes

In [41]:
base_model = InceptionV3(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

96112376/96112376 [==============================] - 1s 0us/step


In [42]:
base_model_1 = InceptionV3(weights='imagenet', include_top=False)

87910968/87910968 [==============================] - 1s 0us/step


In [43]:
videos = os.listdir('/content/gdrive/MyDrive/PART4.2/train')
videos

['_frame0.jpg',
 '_frame1.jpg',
 '_frame2.jpg',
 '_frame3.jpg',
 '_frame4.jpg',
 '_frame5.jpg',
 '_frame6.jpg',
 '_frame7.jpg',
 '_frame8.jpg',
 '_frame9.jpg',
 '_frame10.jpg',
 '_frame11.jpg',
 '_frame12.jpg',
 '_frame13.jpg',
 '_frame14.jpg',
 '_frame15.jpg',
 '_frame16.jpg',
 '_frame17.jpg',
 '_frame18.jpg',
 '_frame19.jpg',
 '_frame20.jpg',
 '_frame21.jpg',
 '_frame22.jpg',
 '_frame23.jpg',
 '_frame24.jpg',
 '_frame25.jpg',
 '_frame26.jpg',
 '_frame27.jpg',
 '_frame28.jpg',
 '_frame29.jpg',
 '_frame30.jpg',
 '_frame31.jpg',
 '_frame32.jpg',
 '_frame33.jpg',
 '_frame34.jpg',
 '_frame35.jpg',
 '_frame36.jpg',
 '_frame37.jpg',
 '_frame38.jpg',
 '_frame39.jpg',
 '_frame40.jpg',
 '_frame41.jpg',
 '_frame42.jpg',
 '_frame43.jpg',
 '_frame44.jpg',
 '_frame45.jpg',
 '_frame46.jpg',
 '_frame47.jpg',
 '_frame48.jpg',
 '_frame49.jpg',
 '_frame50.jpg',
 '_frame51.jpg',
 '_frame52.jpg',
 '_frame53.jpg',
 '_frame54.jpg',
 '_frame55.jpg',
 '_frame56.jpg',
 '_frame57.jpg',
 '_frame58.jpg',
 '_fram

In [44]:
sample_frame = '/content/gdrive/MyDrive/PART4.2/train/_frame1.jpg'

In [45]:
 predictions_df = DataFrame(columns=['Class','Source'])
 predictions_df

,Class,Source


In [ ]:
!mkdir '/content/gdrive/MyDrive/PART4.2/train1'

In [46]:
def save_video_frames(video_path):
    count = 0
    cap = cv2.VideoCapture(video_path)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    names = []
    while(cap.isOpened()):

      frameId = cap.get(1) #current frame number
      ret, frame = cap.read()

      if (ret != True):
          break
      if (frameId % math.floor(frameRate) == 0):
          # storing the frames in a new folder named train_1
          
          filename ='./train' + video_path.replace(SOURCE,'') +"_frame%d.jpg" % count;count+=1
          cv2.imwrite(filename, frame)
    cap.release()
    

In [49]:
def predict(frame):
    # Load the pre-trained Inception v3 model
    model = tf.keras.applications.InceptionV3(weights='imagenet')
    # pred_df = pd.DataFrame(columns=['Class','Source'])
    # Load the image
    img = cv2.imread(frame)

    # Preprocess the image
    img = cv2.resize(img, (299, 299))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    # Run the image through the model to get predictions
    predictions = model.spredict(img)

    # Decode the predictions
    decoded_predictions = tf.keras.applications.imagenet_utils.decode_predictions(predictions)

    preds = []
    # Print the top 5 predictions
    for i in range(5):
        # print(decoded_predictions[0][i][1])
        preds.append({'Class':decoded_predictions[0][i][1],'Source':frame})
        print(decoded_predictions[0][i][1])
        # print((decoded_predictions[0][i][1],frame))


    return preds

In [51]:
def make_dataf(file_path):
  predictions_df = DataFrame(columns=['Class','Source'])
  predictions = []
  for filename in os.listdir(file_path):
    pred = []
    print(filename)

    pred =_predict(os.path.join('./train',filename))
    print(type(pred))

    for prediction in pred:
      predictions_df = predictions_df.append(prediction,ignore_index=True)   
  predictions_df.to_csv('predictionsfile.csv')


In [56]:
save_video_frames('/content/gdrive/MyDrive/PART4.2/videos')
make_dataf('/content/gdrive/MyDrive/PART4.2/train1')

In [57]:
import pandas as pd
datafram = pd.read_csv('predictionsfile.csv')
datafram = datafram[['Class','Source']]
datafram

,Class,Source


In [ ]:
datafram[datafram['Class'] == 'scoreboard']

In [58]:
def predict(frame):
    print(frame)
    # Load the pre-trained Inception v3 model
    model = tf.keras.applications.InceptionV3(weights='imagenet')
    img = cv2.imread(frame)
    

    # Preprocess the image
    img = cv2.resize(img, (299, 299))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    # Run the image through the model to get predictions
    predictions = model.predict(img)

    # Decode the predictions
    decoded_predictions = tf.keras.applications.imagenet_utils.decode_predictions(predictions)

    preds = []
    # Print the top 5 predictions
    for i in range(5):
        # print(decoded_predictions[0][i][1])
        preds.append({'Class':decoded_predictions[0][i][1],'Source':frame})

    return preds

In [59]:
from keras.models import save_model, load_model
save_model(model, '/content/gdrive/MyDrive/PART4.2/inceptionv3.h5')

In [64]:
# Define a function to search for objects in the video
def search_video(query):
    # Loop through each frame in the video
    while True:
        ret, frame = cap.read()

        # Stop the loop if we have reached the end of the video
        if not ret:
            break

        # Preprocess the frame
        img = preprocess_image(frame)
        # Make a prediction using the pre-trained InceptionV3 model
        preds = model.predict(img)
        decoded_preds = keras.applications.inception_v3.decode_predictions(preds, top=1)[0]
        class_name = decoded_preds[0][1]
        confidence = decoded_preds[0][2]

        # Check if the predicted class name contains the query
        if query.lower() in class_name.lower():
            # Display the frame and the predicted class name
            cv2.putText(frame, class_name, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow('frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
